<a href="https://colab.research.google.com/github/kimgw-pl/computer_vision/blob/main/GBC_balls_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'글 소스'   주차별   GBC_balls	 mp4  '카메라 케이스.xlsx'


github에서 yolov5 다운

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 6146, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 6146 (delta 96), reused 109 (delta 69), pack-reused 5980
Receiving objects: 100% (6146/6146), 8.70 MiB | 2.39 MiB/s, done.
Resolving deltas: 100% (4188/4188), done.


yolov5 로 이동한 후 필요한 패키지들을 다운받아줍니다.

In [ ]:
%cd /content/yolov5/
! pip install -r requirements.txt

/content/yolov5
     |████████████████████████████████| 645kB 9.9MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cat /content/gdrive/MyDrive/GBC_balls/GBC_balls.yaml # yaml 파일 확인

train: ../train/images
val: ../valid/images

nc: 2
names: ['B', 'S']

glob을 통해 이미지들의 경로를 다 받아오는 과정

In [ ]:
%cd /
from glob import glob

img_list = glob('/content/gdrive/MyDrive/GBC_balls/export/images/*.png')
print(len(img_list))

/
98


이 이미지를 train과 test로 9:1로 나눠준다.

In [ ]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size=0.1, random_state=1000)

print(len(train_img_list), len(val_img_list))

88 10


In [ ]:
with open('content/gdrive/MyDrive/GBC_balls/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')

with open('content/gdrive/MyDrive/GBC_balls/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list)+'\n')

yaml 파일에서 경로를 적어주는 과정

In [ ]:
import yaml
with open('/content/gdrive/MyDrive/GBC_balls/GBC_balls.yaml', 'r') as f:
  data = yaml.load(f)

print(data)

data['train'] = '/content/gdrive/MyDrive/GBC_balls/train.txt'
data['val'] = '/content/gdrive/MyDrive/GBC_balls/val.txt'

with open('/content/gdrive/MyDrive/GBC_balls/GBC_balls.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 2, 'names': ['B', 'S']}
{'train': '/content/gdrive/MyDrive/GBC_balls/train.txt', 'val': '/content/gdrive/MyDrive/GBC_balls/val.txt', 'nc': 2, 'names': ['B', 'S']}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 50 --data /content/gdrive/MyDrive/GBC_balls/GBC_balls.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name GBC_yolov5s_results

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-64-g5189b3a torch 1.8.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='./models/yolov5s.yaml', data='/content/gdrive/MyDrive/GBC_balls/GBC_balls.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='GBC_yolov5s_results', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/GBC_yolov5s_results', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-05-04 12

In [ ]:
%cd /
from google.colab import files
uploaded = files.upload()

/


Saving GBC_Ball.mp4 to GBC_Ball.mp4


In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/yolov5/runs/train/GBC_yolov5s_results/weights/best.pt --source /content/gdrive/MyDrive/GBC_Ball.mp4